In [1]:
# CÉLULA 1 — Instalar bibliotecas essenciais, se ainda não estiverem disponíveis
!pip install --upgrade pip
!pip install boto3 openai python-dotenv


In [3]:
# CÉLULA 2 — Gera o .env automaticamente a partir dos arquivos enviados

from pathlib import Path
import pandas as pd

# Caminhos esperados dos arquivos (assumindo que estão na raiz do projeto ou /app)
aws_csv_path = Path("/app/WRMELO_accessKeys.csv")
openai_key_path = Path("/app/OPEN AI - CHAVE.md")
env_path = Path("/app/.env")

# Leitura das credenciais AWS
aws_df = pd.read_csv(aws_csv_path)
aws_access_key = aws_df["Access key ID"].iloc[0].strip()
aws_secret_key = aws_df["Secret access key"].iloc[0].strip()

# Leitura da chave OpenAI
openai_key = Path(openai_key_path).read_text().strip()

# Região e bucket fixos
aws_region = "us-east-1"
s3_bucket = "staging-face-text"

# Escrita do .env
env_content = f"""AWS_ACCESS_KEY_ID={aws_access_key}
AWS_SECRET_ACCESS_KEY={aws_secret_key}
AWS_REGION={aws_region}
OPENAI_API_KEY={openai_key}
S3_BUCKET={s3_bucket}
"""

env_path.write_text(env_content)
print("✅ Arquivo .env criado com sucesso em:", env_path)


✅ Arquivo .env criado com sucesso em: /app/.env


In [4]:
# CÉLULA 3 — Carrega o .env e verifica as variáveis obrigatórias

from dotenv import load_dotenv
import os

load_dotenv(dotenv_path="/app/.env")

required_vars = [
    "AWS_ACCESS_KEY_ID",
    "AWS_SECRET_ACCESS_KEY",
    "AWS_REGION",
    "OPENAI_API_KEY",
    "S3_BUCKET"
]

missing = [var for var in required_vars if not os.getenv(var)]
if missing:
    raise EnvironmentError(f"❌ Variáveis faltando no .env: {missing}")

print("✅ Variáveis de ambiente carregadas com sucesso:\n")
for var in required_vars:
    print(f"{var}: {'***' + os.getenv(var)[-4:]}")


✅ Variáveis de ambiente carregadas com sucesso:

AWS_ACCESS_KEY_ID: ***QZPP
AWS_SECRET_ACCESS_KEY: ***pdvz
AWS_REGION: ***st-1
OPENAI_API_KEY: ***Z_UA
S3_BUCKET: ***text


In [10]:
# CÉLULA 4 (REVISADA) — Teste real com AWS e OpenAI adaptado

import boto3
import openai
import os

# AWS S3 — testa acesso ao bucket especificado
try:
    s3 = boto3.client(
        "s3",
        aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
        aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
        region_name=os.getenv("AWS_REGION")
    )
    bucket_name = os.getenv("S3_BUCKET")
    s3.head_bucket(Bucket=bucket_name)
    print(f"✅ Conexão AWS S3 OK — Acesso confirmado ao bucket '{bucket_name}'")
except Exception as e:
    print(f"❌ Falha ao acessar bucket '{bucket_name}':", e)

# OpenAI — nova API, converte para lista
try:
    openai.api_key = os.getenv("OPENAI_API_KEY")
    models = list(openai.models.list())
    print(f"✅ Conexão OpenAI OK — {len(models)} modelos disponíveis")
except Exception as e:
    print("❌ Erro na conexão com OpenAI:", e)


✅ Conexão AWS S3 OK — Acesso confirmado ao bucket 'staging-face-text'
✅ Conexão OpenAI OK — 50 modelos disponíveis


In [11]:
# CÉLULA 5 — Criação do bucket 'staging-face-text' se ele não existir

import botocore.exceptions

bucket_name = os.getenv("S3_BUCKET")

try:
    s3.head_bucket(Bucket=bucket_name)
    print(f"✅ Bucket '{bucket_name}' já existe.")
except botocore.exceptions.ClientError as e:
    error_code = int(e.response['Error']['Code'])
    if error_code == 404:
        print(f"⚠️ Bucket '{bucket_name}' não encontrado. Tentando criar...")
        try:
            s3.create_bucket(Bucket=bucket_name)
            print(f"✅ Bucket '{bucket_name}' criado com sucesso.")
        except Exception as create_err:
            print(f"❌ Falha ao criar bucket '{bucket_name}':", create_err)
    else:
        print(f"❌ Erro ao verificar o bucket '{bucket_name}':", e)


✅ Bucket 'staging-face-text' já existe.
